In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

In [ ]:
# Load YOLOv8 model
model = YOLO("yolov8n.pt")  # Use the appropriate YOLOv8 model file

In [ ]:
# Initialize video capture
cap = cv2.VideoCapture("video.mp4")

In [ ]:
# Define ROI or line for counting
line_position = 300
car_count = 0
tracked_cars = set()

In [ ]:
import cv2
import numpy as np

def get_car_color(roi):
    # Convert ROI to HSV color space
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    
    # Define HSV color ranges
    color_ranges = {
        "Red": [(0, 50, 50), (10, 255, 255), (170, 50, 50), (180, 255, 255)],
        "Orange": [(10, 50, 50), (25, 255, 255)],
        "Yellow": [(25, 50, 50), (35, 255, 255)],
        "Green": [(35, 50, 50), (85, 255, 255)],
        "Blue": [(85, 50, 50), (125, 255, 255)],
        "Purple": [(125, 50, 50), (145, 255, 255)],
        "Pink": [(145, 50, 50), (170, 255, 255)],
        "White": [(0, 0, 200), (180, 30, 255)]
    }
    
    max_pixels = 0
    dominant_color = "Unknown"
    
    for color, ranges in color_ranges.items():
        mask = None
        for i in range(0, len(ranges), 2):
            lower_bound = np.array(ranges[i])
            upper_bound = np.array(ranges[i+1])
            if mask is None:
                mask = cv2.inRange(hsv, lower_bound, upper_bound)
            else:
                mask += cv2.inRange(hsv, lower_bound, upper_bound)
        
        num_pixels = cv2.countNonZero(mask)
        if num_pixels > max_pixels:
            max_pixels = num_pixels
            dominant_color = color
    
    return dominant_color

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    height, width, channels = frame.shape

    # Detect cars using YOLOv8
    results = model(frame)

    for result in results:
        boxes = result.boxes.xyxy.cpu().numpy()  # Get bounding boxes
        confidences = result.boxes.conf.cpu().numpy()  # Get confidences
        class_ids = result.boxes.cls.cpu().numpy()  # Get class IDs

        for i in range(len(boxes)):
            if confidences[i] > 0.5 and class_ids[i] == 2:  # Class ID 2 is for 'car' in COCO dataset
                x, y, x2, y2 = boxes[i]
                x, y, w, h = int(x), int(y), int(x2 - x), int(y2 - y)
                roi = frame[y:y+h, x:x+w]
                car_color = get_car_color(roi)
                label = f"Car {car_color}"
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                    # Generate a unique ID for the car based on its position
                car_id = (x, y, w, h)

                # Check if the car crosses the line
                if y + h > line_position and y < line_position and car_id not in tracked_cars:
                    car_count += 1
                tracked_cars.add(car_id)

    # Draw counting line
    cv2.line(frame, (0, line_position), (width, line_position), (0, 0, 255), 2)
    cv2.putText(frame, f"Count: {car_count}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    cv2.imshow("Frame", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()